<a href="https://colab.research.google.com/github/JorgeAccardi/auscultacion-presa/blob/main/Carga_Datos_Versi%C3%B3n_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import io
import base64
from datetime import datetime
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets

# --- Configuración y estilos ---
instrumentos = [
    "puntos_fijos_mi",
    "puntos_fijos_md",
    "inclinometros",
    "asentamiento",
    "piezometros_electricos",
    "piezometros_casagrande",
    "freatimetros",
    "extensometro"
]
datos_csv = {inst: pd.DataFrame() for inst in instrumentos}
datos_xlsx = {inst: pd.DataFrame() for inst in instrumentos}

def detectar_instrumento(nombre):
    nombre = nombre.lower()
    if "puntosfijos" in nombre or "pf" in nombre:
        if "mi" in nombre:
            return "puntos_fijos_mi"
        elif "md" in nombre:
            return "puntos_fijos_md"
        else:
            return "puntos_fijos_mi"
    elif "incli" in nombre:
        return "inclinometros"
    elif "as" in nombre:
        return "asentamiento"
    elif "pe" in nombre:
        return "piezometros_electricos"
    elif "pcg" in nombre:
        return "piezometros_casagrande"
    elif "frea" in nombre:
        return "freatimetros"
    elif "ext" in nombre:
        return "extensometro"
    return None

# Widgets globales para mantener la UI estable
upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=True,
    description='Subir archivos',
    style={'button_color': 'lightblue'},
    layout=widgets.Layout(width="350px")
)
output_carga = widgets.Output()
output_tabla = widgets.Output()

instrumento_selector = widgets.Dropdown(
    options=instrumentos,
    description='Instrumento:',
    layout=widgets.Layout(width='260px')
)
origen_selector = widgets.Dropdown(
    options=['csv', 'xlsx'],
    description='Origen:',
    layout=widgets.Layout(width='160px')
)
boton_ver = widgets.Button(
    description='👁️ Ver',
    button_style='success',
    icon='eye',
    layout=widgets.Layout(width='100px')
)
boton_descargar = widgets.Button(
    description='💾 Descargar',
    button_style='info',
    icon='download',
    layout=widgets.Layout(width='130px')
)

# --- Función de carga y progreso ---
def cargar_archivos(change):
    with output_carga:
        clear_output(wait=True)
        archivos = upload_widget.value
        if not archivos:
            display(HTML("<div style='color:#b71c1c;font-weight:bold;'>⚠️ No se subió ningún archivo.</div>"))
            return

        barra_progreso = widgets.FloatProgress(
            value=0, min=0, max=100, description='Progreso:', bar_style='info',
            layout=widgets.Layout(width='80%')
        )
        etiqueta_progreso = widgets.Label(value="0% completado")
        display(barra_progreso, etiqueta_progreso)

        total = len(archivos)
        archivos_exitosos = 0

        for i, (nombre_archivo, archivo_info) in enumerate(archivos.items(), start=1):
            try:
                extension = nombre_archivo.split('.')[-1].lower()
                instrumento = detectar_instrumento(nombre_archivo)
                contenido = archivo_info['content']

                if not instrumento:
                    display(HTML(f"<span style='color:#b71c1c;'>❌ Instrumento no reconocido en archivo: {nombre_archivo}</span>"))
                    continue

                if extension == 'csv':
                    try:
                        try:
                            df = pd.read_csv(io.BytesIO(contenido), encoding='utf-8')
                        except UnicodeDecodeError:
                            df = pd.read_csv(io.BytesIO(contenido), encoding='latin-1')
                        if not df.empty:
                            datos_csv[instrumento] = pd.concat([datos_csv[instrumento], df], ignore_index=True)
                            archivos_exitosos += 1
                            display(HTML(f"<span style='color:#388e3c;'>✔️ {nombre_archivo} cargado como CSV ({len(df)} filas)</span>"))
                        else:
                            display(HTML(f"<span style='color:#ffa000;'>⚠️ CSV vacío: {nombre_archivo}</span>"))
                    except Exception as e:
                        display(HTML(f"<span style='color:#b71c1c;'>❌ Error leyendo CSV {nombre_archivo}: {str(e)}</span>"))
                elif extension == 'xlsx':
                    try:
                        df = pd.read_excel(io.BytesIO(contenido))
                        if not df.empty:
                            datos_xlsx[instrumento] = pd.concat([datos_xlsx[instrumento], df], ignore_index=True)
                            archivos_exitosos += 1
                            display(HTML(f"<span style='color:#388e3c;'>✔️ {nombre_archivo} cargado como XLSX ({len(df)} filas)</span>"))
                        else:
                            display(HTML(f"<span style='color:#ffa000;'>⚠️ XLSX vacío: {nombre_archivo}</span>"))
                    except Exception as e:
                        display(HTML(f"<span style='color:#b71c1c;'>❌ Error leyendo XLSX {nombre_archivo}: {str(e)}</span>"))
                else:
                    display(HTML(f"<span style='color:#b71c1c;'>⚠️ Extensión no soportada: {nombre_archivo}</span>"))
            except Exception as e:
                display(HTML(f"<span style='color:#b71c1c;'>❌ Error general en {nombre_archivo}: {str(e)}</span>"))
            porcentaje = (i / total) * 100
            barra_progreso.value = porcentaje
            etiqueta_progreso.value = f"{porcentaje:.0f}% completado"

        barra_progreso.bar_style = 'success'
        etiqueta_progreso.value = f"✅ {archivos_exitosos}/{total} archivos procesados exitosamente"
        resumen = "<ul>"
        for inst in instrumentos:
            csv_count = len(datos_csv[inst])
            xlsx_count = len(datos_xlsx[inst])
            if csv_count > 0 or xlsx_count > 0:
                resumen += f"<li><b>{inst}</b>: {csv_count} filas CSV, {xlsx_count} filas XLSX</li>"
        resumen += "</ul>"
        display(HTML(f"<div style='margin-top:10px;'><b>Resumen de datos:</b>{resumen}</div>"))

# --- Función para mostrar tabla o mensaje ---
def ver_datos(b):
    with output_tabla:
        clear_output(wait=True)
        instrumento = instrumento_selector.value
        origen = origen_selector.value
        df = datos_csv[instrumento] if origen == 'csv' else datos_xlsx[instrumento]
        if df.empty:
            display(HTML("<div style='color:#b71c1c;font-weight:bold;'>⚠️ No hay datos disponibles para el instrumento y origen seleccionados.</div>"))
        else:
            display(HTML(f"<div style='margin-bottom:10px;'><b>{instrumento.replace('_',' ').title()} ({origen.upper()})</b> - <span style='color:#388e3c'>Filas: {len(df)} | Columnas: {len(df.columns)}</span></div>"))
            display(df.head(5))  # AJUSTE: solo 5 registros

# --- Función para descargar con enlace bonito ---
def descargar_datos(b):
    with output_tabla:
        clear_output(wait=True)
        instrumento = instrumento_selector.value
        origen = origen_selector.value
        df = datos_csv[instrumento] if origen == 'csv' else datos_xlsx[instrumento]
        if df.empty:
            display(HTML("<div style='color:#b71c1c;font-weight:bold;'>⚠️ No hay datos para descargar.</div>"))
            return
        fecha_actual = datetime.now().strftime("%Y%m%d_%H%M%S")
        extension = 'csv' if origen == 'csv' else 'xlsx'
        nombre_archivo = f"{instrumento}_{origen}_{fecha_actual}.{extension}"
        buffer = io.BytesIO()
        try:
            if extension == 'csv':
                df.to_csv(buffer, index=False, encoding='utf-8')
                mime = "text/csv"
            else:
                # openpyxl es necesario para xlsx, pero en Colab/Jupyter viene instalado
                df.to_excel(buffer, index=False, engine='openpyxl')
                mime = "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
            buffer.seek(0)
            b64 = base64.b64encode(buffer.read()).decode()
            # NOTA: El enlace debe estar en una sola línea sin saltos para funcionar bien
            html = f"""
            <div style='padding:12px 18px;background:#e3f2fd;border:1.5px solid #2196f3;border-radius:7px;max-width:420px;'>
                <b>Descarga lista:</b><br>
                <span style='color:#1565c0'><b>{nombre_archivo}</b></span><br>
                <a download="{nombre_archivo}" href="data:{mime};base64,{b64}" target="_blank" style="display:inline-block;margin-top:10px;padding:10px 18px;background:#2196f3;color:white;text-decoration:none;border-radius:4px;font-weight:bold;">
                   📥 Descargar archivo
                </a>
            </div>
            """
            display(HTML(html))
            display(HTML("<span style='color:#388e3c;'>✔️ Haz clic en el botón para guardar el archivo en tu PC.</span>"))
        except Exception as e:
            display(HTML(f"<span style='color:#b71c1c;'>❌ Error al preparar la descarga: {str(e)}</span>"))

# --- Conectar eventos (solo una vez) ---
upload_widget.observe(cargar_archivos, names='value')
boton_ver.on_click(ver_datos)
boton_descargar.on_click(descargar_datos)

# --- Mostrar interfaz visual limpia y separada ---
display(HTML("""
<div style='margin-bottom:15px;'>
    <h2 style='color:#1976d2;margin:0 0 4px 0;'>📈 Sistema de gestión de datos de instrumentos</h2>
    <span style='color:#555;'>Carga, visualización y descarga de archivos CSV/XLSX</span>
</div>
"""))
display(HTML("<b>1. Subí tus archivos CSV/XLSX:</b>"))
display(upload_widget)
display(output_carga)
display(HTML("<hr style='margin:20px 0 10px 0;'>"))
display(HTML("<b>2. Seleccioná instrumento y origen de datos:</b>"))
display(widgets.HBox([instrumento_selector, origen_selector, boton_ver, boton_descargar]))
display(output_tabla)

FileUpload(value={}, accept='.csv,.xlsx', description='Subir archivos', layout=Layout(width='350px'), multiple…

Output()

Output()